In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, datasets
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import classification_report,accuracy_score
import pandas as pd
from sklearn.datasets import load_iris

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow
import keras
from keras.layers import *
from keras.models import *
from keras.preprocessing import image

Using TensorFlow backend.


In [2]:
import os
from keras.preprocessing import image

global base_dir, image_paths, target_size
base_dir = './dataset'
images_dir = os.path.join(base_dir,'image')
label_filename = 'label.csv'

image_paths = [os.path.join(images_dir, l) for l in os.listdir(images_dir)]
target_size = None
# Opening the label file
label_file = open(os.path.join(base_dir, label_filename), 'r')
# Reading the lines
label_lines = label_file.readlines()
#print(image_paths)
tumor_labels = {line.split(',')[0] : line.split(',')[1] for line in label_lines[1:]}
for k, v in tumor_labels.items():
    tumor_labels[k] = v.replace('\n', '')
#tumor_labels[:,1] = {line.replace('\n', '') for line in tumor_labels[1:]}
for k in tumor_labels:
    if 'no_tumor' in tumor_labels[k]:
        tumor_labels[k] = 0
    else:
        tumor_labels[k] = 1
    #tumor_labels[k].contains() == 
#print(tumor_labels)

if os.path.isdir(images_dir):
    all_features = []
    #         all_features = []
    all_labels = []
    for img_path in image_paths:
         #No need for this 
            # file_name= img_path.split('/')[-1]
        #file_name= img_path.split('.')[1].split('/')[-1] + '.jpg'
        file_name= img_path.split('\\')[-1]
        # load image
        img = image.img_to_array(image.load_img(img_path, color_mode='grayscale'))
        #img = image.img_to_array(image.load_img(img_path, color_mode='grayscale', target_size=target_size, interpolation='bicubic'))
        img = np.reshape(img, (512,512, 1))
        # Getting rid of rgb data to greyscale
        #img = np.squeeze(img)
        if img is not None:
            all_features.append(img)
            all_labels.append(tumor_labels[file_name])


In [3]:
x = np.array(all_features)
y = np.array(all_labels)
# print(xtrain.shape)
#x = np.squeeze(x)
#print(xtrain.shape)
x /= 255

xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.20, random_state=10) 

#img = mpimg.imread('your_image.png')
#imgplot = plt.imshow(all_features[1], cmap='Greys_r')



In [12]:
print(xtrain.shape)
print(ytrain.shape)

(2400, 512, 512, 1)
(2400,)


In [4]:
cnn_model = Sequential()
cnn_model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(512,512,1)))
cnn_model.add(Conv2D(64, (3,3), activation='relu'))
cnn_model.add(MaxPooling2D(pool_size=(2,2)))
cnn_model.add(Dropout(0.25))

cnn_model.add(Conv2D(64, (3,3), activation='relu'))
cnn_model.add(MaxPooling2D(pool_size=(2,2)))
cnn_model.add(Dropout(0.25))

cnn_model.add(Conv2D(128, (3,3), activation='relu'))
cnn_model.add(MaxPooling2D(pool_size=(2,2)))
cnn_model.add(Dropout(0.25))

cnn_model.add(Flatten())
cnn_model.add(Dense(64, activation='relu'))
cnn_model.add(Dropout(0.25))
cnn_model.add(Dense(1, activation='sigmoid'))

cnn_model.compile(loss=keras.losses.binary_crossentropy, optimizer='adam', metrics=['accuracy'])

In [18]:
cnn_model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 510, 510, 32)      320       
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 508, 508, 64)      18496     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 254, 254, 64)      0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 254, 254, 64)      0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 252, 252, 64)      36928     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 126, 126, 64)      0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 126, 126, 64)     

In [5]:
final_model = cnn_model.fit(xtrain, ytrain, batch_size=10, epochs=10, validation_data=(xtest, ytest))

Train on 2400 samples, validate on 600 samples
Epoch 1/10
2400/2400 [==============================] - 57s 24ms/step - loss: 0.4416 - accuracy: 0.8417 - val_loss: 0.2983 - val_accuracy: 0.8550
Epoch 2/10
2400/2400 [==============================] - 49s 21ms/step - loss: 0.2781 - accuracy: 0.8550 - val_loss: 0.2695 - val_accuracy: 0.8600
Epoch 3/10
2400/2400 [==============================] - 49s 20ms/step - loss: 0.2215 - accuracy: 0.9008 - val_loss: 0.2574 - val_accuracy: 0.9017
Epoch 4/10
2400/2400 [==============================] - 49s 21ms/step - loss: 0.1692 - accuracy: 0.9333 - val_loss: 0.2011 - val_accuracy: 0.9383
Epoch 5/10
2400/2400 [==============================] - 49s 21ms/step - loss: 0.1081 - accuracy: 0.9529 - val_loss: 0.2271 - val_accuracy: 0.9383
Epoch 6/10
2400/2400 [==============================] - 48s 20ms/step - loss: 0.0811 - accuracy: 0.9667 - val_loss: 0.2457 - val_accuracy: 0.9333
Epoch 7/10
2400/2400 [==============================] - 48s 20ms/step - loss: